In [ ]:
# # 한국어 전처리를 위한
!pip install konlpy
# !pip install Twitter
# !pip install tqdm
!pip install gensim
# !pip install --upgrade jupyter ipywidgets

In [7]:
import os # 운영 체제 관련 작업을 수행하기 위한 Python 표준 라이브러리 중 하나 -> 자바 환경 변수를 설정하기 위해 사용
import pandas as pd # 데이터 조작 및 분석을 위한 파이썬 라이브러리 -> 데이터를 데이터프레임으로 로드하고 조작하는 데 사용
import numpy as np # 배열 및 수치 연산을 수행하기 위한 라이브러리 -> 다차원 배열 및 수학적 함수를 다루는 데 사용
from konlpy.tag import Okt # Konlpy 라이브러리에서 제공하는 한국어 형태소 분석기 중 하나 ->  Okt를 초기화하는 데 사용
from gensim.models import Word2Vec # Gensim 라이브러리에서 제공하는 Word2Vec 모델을 생성하기 위한 클래스 -> 단어를 고차원 벡터로 표현하여 단어 간 유사성 및 의미 관계를 파악
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
# 자바 설치 경로 설정
os.environ["JAVA_HOME"] = "C:\Program Files\Java\jdk-11"

In [4]:
# 데이터를 데이터프레임으로 로드
book_data_vector = pd.read_csv('../Model_test/book_data_vector.csv', encoding='utf-8')# 데이터 파일 경로
book_data_vector

,SEQ_NO,ISBN_THIRTEEN_NO,VLM_NM,TITLE_NM,AUTHR_NM,PUBLISHER_NM,PBLICTE_DE,ADTION_SMBL_NM,PRC_VALUE,IMAGE_URL,...,KDC_NM,TITLE_SBST_NM,AUTHR_SBST_NM,TWO_PBLICTE_DE,INTNT_BOOKST_BOOK_EXST_AT,PORTAL_SITE_BOOK_EXST_AT,ISBN_NO,BOOK_INTRCN_CN_clear,tokenized_data,doc2vec_vector
0,6352228,9791156759270,NaN,너에게 목소리를 보낼게 달빛천사 성우 이용신의 첫 번째 에세이,이용신 지은이,푸른숲,NaN,NaN,160000.0,httpsimagealadincokrproduct284158coverk6528351...,...,NaN,NaN,NaN,20211203.0,NaN,NaN,NaN,방영 애니메이션 달빛천사 주인공 루나 풀문 역 보석 같다 추억 선물 성우 이용신 첫...,"['방영', '애니메이션', '달빛천사', '주인공', '루나', '풀문', '역'...",b'\x16E\x9d=\xe8\xe0\xe2=\xfe\xed\xac\xbdX\xfd...
1,6352229,9791168120877,NaN,일기에도 거짓말을 쓰는 사람 99년생 시인의 자의식 과잉 에세이,차도하 지은이,위즈덤하우스,NaN,NaN,158000.0,httpsimagealadincokrproduct2841466coverk202835...,...,NaN,NaN,NaN,20211206.0,NaN,NaN,NaN,그렇다 나 말 것 말 침착하다 사랑 차도 시인 첫 에세이 새 도발 작품 문단 대주 ...,"['그렇다', '나', '말', '것', '말', '침착하다', '사랑', '차도'...",b'\n\x8b\xa1=S\xdd\xeb=?E\xb4\xbdl\x9e\x03=\x0...
2,6352230,9791168120839,NaN,본격 한중일 세계사 12 임오군란과 통킹 위기,굽시니스트 지은이,위즈덤하우스,NaN,NaN,148000.0,httpsimagealadincokrproduct2841447coverk402835...,...,NaN,NaN,NaN,20211201.0,NaN,NaN,NaN,한중일 관계 결정 분기점 임오군란 막후 러시아 세력 확장 경계 청 친 결 미라 계책...,"['한중일', '관계', '결정', '분기점', '임오군란', '막후', '러시아'...",b'\x19\xb7\x87=\xeb!\xc1=\xec\xb5\x8f\xbd\xbb\...
3,6352231,9791168120846,NaN,즉시 기분을 바꿔드립니다 신기하게 마음이 편해지는 응급 처방전,올리비아 레메스 지은이 김잔디 옮긴이,위즈덤하우스,NaN,NaN,140000.0,httpsimagealadincokrproduct2841430coverk892835...,...,NaN,NaN,NaN,20211201.0,NaN,NaN,NaN,누구 기분 구급상자 필요하다 하나 하나 차근차근 좋다 기분 건강하다 마음 법 책,"['누구', '기분', '구급상자', '필요하다', '하나', '하나', '차근차근...",b'j\xf5\x9f=\x95Z\xea=|\'\xaa\xbd\xa2\x92\xf2<...
4,6352232,9791168120747,NaN,오늘도 리추얼 음악 나에게 선물하는 시간,정혜윤 지은이,위즈덤하우스,NaN,NaN,150000.0,httpsimagealadincokrproduct2841380coverk202835...,...,NaN,NaN,NaN,20211201.0,NaN,NaN,NaN,일상 사람 이야기 오늘 리추얼 시리즈 위즈 덤 하우스 간 오늘 리추얼 시리즈 음악 ...,"['일상', '사람', '이야기', '오늘', '리추얼', '시리즈', '위즈', ...",b'\xe7S\xac=&S\xf3=r2\xc3\xbd\xce0\n=z\xdd\x95...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1997,6354246,9791164670871,NaN,학교 서클대화가 필요해,손연일 심선화 장경아 지은이,북트리,NaN,NaN,160000.0,httpsimagealadincokrproduct2837599coverk262835...,...,NaN,학교서클대화가필요해,손연일심선화장경아지은이,20211125.0,Y,Y,NaN,평화롭다 학교 공동체 여정 친절하다 안내서 관계 회복하다 배움 평화롭다 학교 공동체...,"['평화롭다', '학교', '공동체', '여정', '친절하다', '안내서', '관계...",b'\xa6h\x95=f\x80\xd6=\xbf(\x9e\xbd?\xea\xe6<\...
1998,6354247,9788994229003,NaN,People Make City,이상환 지은이,디자인로커스,NaN,NaN,150000.0,httpsimagealadincokrproduct141865cover89942290...,...,NaN,peoplemakecity,이상환지은이,20111121.0,Y,Y,NaN,따뜻하다 성북동 만들기 피블 메이크 시티 시리즈 대상 지역 문화 예술 개발 방법론 ...,"['따뜻하다', '성북동', '만들기', '피블', '메이크', '시티', '시리즈...",b'\xa1m\xb0=\xee\xcb\xf9=V_\xbb\xbd\xb4\\\x0e=...
1999,6354248,9788994027203,NaN,동방해경표,김려 지음,미디어버스,NaN,NaN,NaN,NaN,...,NaN,동방해경표,김려지음,NaN,Y,Y,NaN,,"['', '']","b""\xbf\x9a\xc7=\xfe\xab\r>2\xbb\xd3\xbd\x91\x0..."
2000,6354249,9791156759126,NaN,큰글자도서 그렇다면 정상입니다,하지현 지은이,푸른숲,NaN,NaN,360000.0,httpsimagealadincokrproduct2834689coverk292835...,...,NaN,큰글자도서그렇다면정상입니다,하지현지은이,20211028.0,Y,Y,NaN,심야 치유 식당 사랑 결코 엄마 빈틈 아이 등 통해 사람 문제 진단 그 적합하다 해...,"['심야', '치유', '식당', '사랑', '결코', '엄마', '빈틈', '아이...","b""zt\xb2=5\xa7\xf0=\x9c3\xbc\xbd\xdfx\x10=\x94..."


In [10]:
# 저장된 모델을 불러오기
model = Word2Vec.load("../Model_test/book_word2vec_model")

In [11]:
#Vector 형태로 변환한 단어 리스트 조회
model.wv.key_to_index

{'': 0,
 '있다': 1,
 '수': 2,
 '문제': 3,
 '학습': 4,
 '개념': 5,
 '구성': 6,
 '수록': 7,
 '책': 8,
 '수학': 9,
 '및': 10,
 '통해': 11,
 '주': 12,
 '것': 13,
 '독해': 14,
 '유형': 15,
 '연습': 16,
 '내용': 17,
 '등': 18,
 '다양하다': 19,
 '기출': 20,
 '주문': 21,
 '이해': 22,
 '이': 23,
 '교과': 24,
 '개': 25,
 '공부': 26,
 '별': 27,
 '단어': 28,
 '단계': 29,
 '과정': 30,
 '교재': 31,
 '속': 32,
 '방법': 33,
 '이야기': 34,
 '아이': 35,
 '시험': 36,
 '제공': 37,
 '위': 38,
 '시리즈': 39,
 '수능': 40,
 '어휘': 41,
 '완성': 42,
 '모의고사': 43,
 '실전': 44,
 '생활': 45,
 '핵심': 46,
 '연산': 47,
 '정리': 48,
 '재미있다': 49,
 '설명': 50,
 '어떻다': 51,
 '지문': 52,
 '작품': 53,
 '기본': 54,
 '영역': 55,
 '대한': 56,
 '학년': 57,
 '계산': 58,
 '적용': 59,
 '대비': 60,
 '권': 61,
 '만화': 62,
 '기초': 63,
 '그': 64,
 '필수': 65,
 '교과서': 66,
 '문장': 67,
 '저자': 68,
 '분석': 69,
 '체계': 70,
 '문학': 71,
 '집': 72,
 '가지': 73,
 '각': 74,
 '준비': 75,
 '위해': 76,
 '초등': 77,
 '법': 78,
 '풀이': 79,
 '교육': 80,
 '가능하다': 81,
 '같다': 82,
 '아니다': 83,
 '작가': 84,
 '해설': 85,
 '자료': 86,
 '글쓰기': 87,
 '확인': 88,
 '쓰기': 89,
 '중심': 90,
 '집중': 91,
 '

In [30]:
# "문제"와 유사한 단어 10개 찾기
similar_words = model.wv.most_similar("동물", topn=20)

# 유사한 단어와 그 유사도 출력
for word, similarity in similar_words:
    print(f"{word}: {similarity}")

사회: 0.9983928203582764
없다: 0.9983277320861816
하나: 0.9982373118400574
인간: 0.9981957674026489
시작: 0.9981409311294556
삶: 0.9981324076652527
역사: 0.9981170892715454
친구: 0.9981061816215515
데: 0.9981037378311157
가장: 0.998099684715271
학년: 0.9980965256690979
시대: 0.9980760216712952
지역: 0.9980644583702087
표현: 0.9980461001396179
더: 0.9980391263961792
기존: 0.9980372190475464
활동: 0.9980266690254211
소개: 0.9980061650276184
좋다: 0.9980002641677856
어린이: 0.9979974627494812
